<a href="https://colab.research.google.com/github/Eieiz007/Deep-learning/blob/main/pytorch_ANN_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import data

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# โหลด Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

In [6]:
X[:6]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4]])

Splitting data

In [8]:
# แบ่งเป็น Train / Validation / Test 70:15:15
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [10]:
# Standardization (ทำให้ค่าอยู่ในช่วง -3 ถึง 3)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [13]:
# แปลงเป็น Tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_val = torch.tensor(y_val, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

<ipython-input-13-4dc2aa61ec18>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train, dtype=torch.float32)
<ipython-input-13-4dc2aa61ec18>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val = torch.tensor(X_val, dtype=torch.float32)
<ipython-input-13-4dc2aa61ec18>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_test, dtype=torch.float32)
<ipython-input-13-4dc2aa61ec18>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTen

In [20]:
class CustomANN(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, hidden3_size, output_size):
        super(CustomANN, self).__init__()

        # กำหนด Weight และ Bias เอง
        self.W1 = nn.Parameter(torch.randn(input_size, hidden1_size) * 0.01) #4 > 10
        self.b1 = nn.Parameter(torch.zeros(hidden1_size))

        self.W2 = nn.Parameter(torch.randn(hidden1_size, hidden2_size) * 0.01) #10 > 5
        self.b2 = nn.Parameter(torch.zeros(hidden2_size))

        self.W3 = nn.Parameter(torch.randn(hidden2_size, hidden3_size) * 0.01) #5 > 3
        self.b3 = nn.Parameter(torch.zeros(hidden3_size))

        self.W4 = nn.Parameter(torch.randn(hidden3_size, output_size) * 0.01) # 3 > 3
        self.b4 = nn.Parameter(torch.zeros(output_size))

    def forward(self, x):
        x = torch.matmul(x, self.W1) + self.b1 #build W^Tx + b of input layer within hidden layer1
        x = torch.relu(x)

        x = torch.matmul(x, self.W2) + self.b2
        x = torch.relu(x)

        x = torch.matmul(x, self.W3) + self.b3
        x = torch.relu(x)

        x = torch.matmul(x, self.W4) + self.b4
        return x  # ไม่ใช้ Softmax เพราะ CrossEntropyLoss() จัดการให้แล้ว


In [27]:
class ANN_Linear(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, hidden3_size, output_size):
        super(ANN_Linear, self).__init__()
        self.input = nn.Linear(input_size, hidden1_size)
        self.h1 = nn.Linear(hidden1_size, hidden2_size)
        self.h2 = nn.Linear(hidden2_size, hidden3_size)
        self.output = nn.Linear(hidden3_size, output_size)

    def forward(self, x):
        x = torch.relu(self.input(x))
        x = torch.relu(self.h1(x))
        x = torch.relu(self.h2(x))
        x = self.output(x)  # Output Layer
        return x

In [28]:
# กำหนด Hyperparameters
input_size = 4
hidden1_size = 10
hidden2_size = 5
hidden3_size = 3
output_size = 3
learning_rate = 0.001
epochs = 100

# สร้างโมเดล
# model = CustomANN(input_size, hidden1_size, hidden2_size, hidden3_size, output_size)
model = ANN_Linear(input_size, hidden1_size, hidden2_size, hidden3_size, output_size)

# Loss Function → CrossEntropyLoss ใช้กับ Classification
criterion = nn.CrossEntropyLoss() #softmax

# Optimizer → Adam (นิยมใช้)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [29]:
train_losses, val_losses = [], []  # เก็บค่าของ Loss

for epoch in range(epochs):
    model.train()  # เปิดโหมด Training
    optimizer.zero_grad()  # ล้างค่า Gradient เก่า

    #forward pass
    outputs = model(X_train)  # Forward Pass
    loss = criterion(outputs, y_train)  # คำนวณ Loss

    #backward pass
    loss.backward()  # คำนวณ Gradient autograd
    optimizer.step()  # อัปเดตค่า W และ b

    # คำนวณ Loss บน Validation Set
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val)

    train_losses.append(loss.item())
    val_losses.append(val_loss.item())

    if epoch % 10 == 0:
        print(f"Epoch [{epoch}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")


Epoch [0/100], Train Loss: 1.1066, Val Loss: 1.3464
Epoch [10/100], Train Loss: 1.0944, Val Loss: 1.3228
Epoch [20/100], Train Loss: 1.0824, Val Loss: 1.3005
Epoch [30/100], Train Loss: 1.0700, Val Loss: 1.2788
Epoch [40/100], Train Loss: 1.0565, Val Loss: 1.2563
Epoch [50/100], Train Loss: 1.0410, Val Loss: 1.2323
Epoch [60/100], Train Loss: 1.0231, Val Loss: 1.2058
Epoch [70/100], Train Loss: 1.0037, Val Loss: 1.1777
Epoch [80/100], Train Loss: 0.9850, Val Loss: 1.1510
Epoch [90/100], Train Loss: 0.9672, Val Loss: 1.1265
